## 1. Import

In [ ]:
pip install mlflow

In [ ]:
!pip install ultralytics

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import os,json
import glob,shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.backend import clear_session
from keras.optimizers import Adam

import mlflow
import mlflow.keras

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.121 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (4 CPUs, 25.5 GB RAM, 24.2/166.8 GB disk)


## 2. 전처리

In [5]:
len(os.listdir('/content/drive/MyDrive/2023/빅프로젝트/data/0609/images/train')), len(os.listdir('/content/drive/MyDrive/2023/빅프로젝트/data/0609/labels/train'))

(300, 300)

In [6]:
len(os.listdir('/content/drive/MyDrive/2023/빅프로젝트/data/0609/images/val')), len(os.listdir('/content/drive/MyDrive/2023/빅프로젝트/data/0609/labels/val'))

(80, 80)

In [7]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 148MB/s]


In [8]:
print(type(model.names), len(model.names))
print(model.names)

<class 'dict'> 80
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell p

In [9]:
import yaml

label_dic = {0:'chair',1:'bike',2:'fan',3:'tv'}

In [ ]:
with open('/content/drive/MyDrive/2023/빅프로젝트/MLOps/class.yaml', 'w') as f:
    data = {
        'path': '/content/drive/MyDrive/2023/빅프로젝트/data/0609',
        'train':'images/train',
        'val': 'images/val',
        'nc': len(label_dic),
        'names': label_dic,
    }
    yaml.safe_dump(data, f)

In [ ]:
#model.train(data='/content/drive/MyDrive/2023/빅프로젝트/MLOps/0616_class_gh.yaml', epochs=100,patience=30,batch=32,imgsz=416)

In [ ]:
import torch

# GPU를 사용할 수 있는지 확인
if torch.cuda.is_available():
    # 특정 GPU 선택 (예: 첫 번째 GPU)
    torch.cuda.set_device(0)
    print(f'Using GPU: {torch.cuda.get_device_name(0)}')
else:
    print('CUDA is not available. Using CPU.')


Using GPU: Tesla V100-SXM2-16GB


## 3. Tracking 준비
* mlflow uri를 setting 해야 모델 로딩도 가능합니다.

In [10]:
mlflow_uri = "http://13.124.68.167:5000/"
mlflow.set_tracking_uri(mlflow_uri)

In [15]:
#exp_id = mlflow.create_experiment("0623")
exp_id = 6
exp_id

6

In [19]:
myname = 'worldpapa2'
model_name = 'test_detail' #의자 소분류 모델이다

In [17]:
## 콜백 함수
def on_fit_epoch_end(trainer):
    if mlflow:
        metrics_dict = {f"{re.sub('[()]', '', k)}": float(v) for k, v in trainer.metrics.items()}
        mlflow.log_metrics(metrics=metrics_dict, step=trainer.epoch)

## 4-1. (택1) MLflow 실행 시작

In [18]:
# MLflow 실행 시작
with mlflow.start_run(experiment_id=exp_id, run_name=myname):

    ## 학습 전

    # YOLO 모델 불러오기
    model = YOLO('yolov8n.pt')

    # 하이퍼파라미터 로깅
    epochs = 2 #100
    patience = 3 #30
    batch_size = 32
    imgsz = 416
    data_path = '/content/drive/MyDrive/2023/빅프로젝트/MLOps/class.yaml'

    mlflow.log_param("epochs", epochs)
    mlflow.log_param("patience", patience)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("imgsz", imgsz)
    mlflow.log_param("data_path", data_path)

    model.add_callback("on_fit_epoch_end",on_fit_epoch_end)

    ## 모델 학습
    results = model.train(data=data_path, epochs=epochs, patience=patience, batch=batch_size, imgsz=imgsz)

    ## 학습 후

    # 모델 메트릭 로깅
    mlflow.log_artifacts('/content/runs/detect/train')

    # 베스트 모델 불러오기
    checkpoint = torch.load('/content/runs/detect/train/weights/best.pt', map_location='cpu')
    best_model = checkpoint.get('model') #, checkpoint)

    # best.pt를 모델 폴더 안에 넣어줘야함!
    mlflow.log_artifact('/content/runs/detect/train/weights/best.pt', artifact_path='best_model')

    # MLflow에 모델 로깅 및 등록된 모델로 등록
    mlflow.pytorch.log_model(best_model, "best_model", registered_model_name=model_name)

Ultralytics YOLOv8.0.121 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/2023/빅프로젝트/MLOps/class.yaml, epochs=1, patience=3, batch=32, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=F

## 4-2. (택1) 모델 돌린 후 저장

In [ ]:
with mlflow.start_run(experiment_id=exp_id, run_name=myname):

    # 모델 불러오기
    checkpoint = torch.load('/content/runs/detect/train/weights/best.pt', map_location='cpu')
    best_model = checkpoint.get('model') #, checkpoint)

    # 모델 메트릭 로깅
    mlflow.log_artifacts('/content/runs/detect/train')

    # best.pt를 모델 폴더 안에 넣어줘야함!
    mlflow.log_artifact('/content/runs/detect/train/weights/best.pt', artifact_path='best_model')

    # MLflow에 모델 로깅 및 등록된 모델로 등록
    mlflow.pytorch.log_model(best_model, "best_model", registered_model_name=model_name)

## 5-1. (택1) 모델 로컬(코랩)에 불러오기 : 스테이지로 불러오기

In [41]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.tracking.artifact_utils import _download_artifact_from_uri

# MLflow 클라이언트 생성
client = MlflowClient()

# 모델 이름과 알리아스(여기서는 "production")를 사용하여 모델의 최신 버전 정보를 가져옴
model_name = "yolo_0621"
latest_version_info = client.get_latest_versions(model_name, stages=["Staging"])

# 최신 버전의 소스(artifact) URI를 가져옴
if latest_version_info:
    source_uri = latest_version_info[0].source
    print(source_uri)
    # source_uri를 사용하여 특정 아티팩트를 다운로드
    #artifact_path = "/best_model/best.pt"
    artifact = _download_artifact_from_uri(
        artifact_uri=source_uri,
        output_path= "/content" #artifact_path
    )

    # `artifact` 변수에는 다운로드된 파일의 로컬 경로가 있습니다.
else:
    print("No model found")

[<ModelVersion: aliases=[], creation_timestamp=1687312512738, current_stage='Staging', description='', last_updated_timestamp=1687312580464, name='yolo_0621', run_id='76183807d482458484bcea80d267f23f', run_link='', source='mlflow-artifacts:/4/76183807d482458484bcea80d267f23f/artifacts/best_model', status='READY', status_message='', tags={}, user_id='', version='3'>]
mlflow-artifacts:/4/76183807d482458484bcea80d267f23f/artifacts/best_model


## 5-2. (택1) 모델 로컬(코랩)에 불러오기 : 버전으로 불러오기

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.tracking.artifact_utils import _download_artifact_from_uri

# MLflow 클라이언트 생성
client = MlflowClient()

# 모델 이름과 버전 지정
model_name = "yolo_0621"
model_version = 3

# 모델의 다운로드 URI 가져오기
source_uri = client.get_model_version_download_uri(model_name, model_version)
print(source_uri)

# 다운로드 URI를 사용하여 모델 다운로드
artifact = _download_artifact_from_uri(
    artifact_uri=source_uri,
    output_path="/content"  # 저장할 경로 지정
)

# `artifact` 변수에는 다운로드된 파일의 로컬 경로가 있습니다.


In [22]:
yolo_model = YOLO('/content/best_model/best.pt')

In [ ]:
result = yolo_model.predict(source = '/content/drive/MyDrive/2023/빅프로젝트/data/0609/images/test', save=True)

## 6. (Optional) 모델 > 버전 설명 추가

http://13.124.68.167:5000/#/models/yolo_0621/versions/3

### description 추가

In [44]:
# 설명 추가
client.update_model_version(
    name=model_name,
    version=str(model_version),  # 정수를 문자열로 변환
    description="욜로 버전3 설명 테스트"
)

<ModelVersion: aliases=[], creation_timestamp=1687312512738, current_stage='Staging', description='욜로 버전3 설명 테스트', last_updated_timestamp=1687486702701, name='yolo_0621', run_id='76183807d482458484bcea80d267f23f', run_link='', source='mlflow-artifacts:/4/76183807d482458484bcea80d267f23f/artifacts/best_model', status='READY', status_message='', tags={'description': '욜로 버전3 태그 테스트'}, user_id='', version='3'>

### tag 추가

In [43]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

model_name = "yolo_0621"
model_version = 3

# 태그 추가
client.set_model_version_tag(
    name=model_name,
    version=str(model_version),
    key="description",
    value="욜로 버전3 태그 테스트"
)

### Schema 추가 (생략)

In [ ]:
# # 스키마 추가

# from mlflow.types import Schema, ColSpec

# # Schema 정의
# schema = Schema([
#     ColSpec("integer", "age"),
#     ColSpec("string", "name"),
# ])

# # 로그를 시작하고 스키마를 로그에 기록
# with mlflow.start_run():
#     mlflow.log_param("my", "param")
#     mlflow.log_metric("score", 100)
#     mlflow.pyfunc.log_model("model", loader_module="mymodule", data_path="mypath", schema=schema)
